In [2]:
import pandas as pd

df = pd.read_csv('./../../../text_emotion.csv')
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [3]:
#converting 

from io import StringIO
col = ['tweet_id','sentiment', 'content']
df = df[col]
df = df[pd.notnull(df['content'])]
# df = df.head(10000)
df.columns = ['tweet_id','sentiment', 'content']
df['category_id'] = df['sentiment'].factorize()[0]
category_id_df = df[['sentiment', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'sentiment']].values)
df.head()

,tweet_id,sentiment,content,category_id
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,0
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,1
2,1956967696,sadness,Funeral ceremony...gloomy friday...,1
3,1956967789,enthusiasm,wants to hang out with friends SOON!,2
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,3


In [4]:
import re
from nltk.corpus import stopwords
engStopWords = stopwords.words("english")

def removeIrrelvantInfo(df,replace_user_reference='<user>',replace_webpage='<url>',replace_hashtag='<hashtag>'):
    regex_user_reference = re.compile('@[a-zA-Z0-9]+')
    references = []
#     for entry in df['content']:
#         references.append(regex_references.findall(entry))
    df['content'] = df['content'].str.replace(regex_user_reference,replace_user_reference)

    regex_websites = re.compile('http://[www.]*[a-zA-Z0-9]+.[a-z]+/[a-zA-Z0-9//]*')
    websites = []
#     for entry in df['content']:
#         regex_websites.findall(entry)
    df['content'] = df['content'].str.replace(regex_websites,replace_webpage)
    regex_extrachars = re.compile('[\.\:]+')
    df['content'] = df['content'].str.replace(regex_extrachars,'')
    df['content'] = df['content'].str.lower()
    df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in engStopWords]))
    df['content'] = df['content'].str.replace(r"n't", " not ")
    df['content'] = df['content'].str.replace(r"?", " ? ")
    df['content'] = df['content'].str.replace(r"!", " ! ")
    return df

In [5]:
print(df.head())
df = removeIrrelvantInfo(df)
print(df.head())

     tweet_id   sentiment                                            content  \
0  1956967341       empty  @tiffanylue i know  i was listenin to bad habi...   
1  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...   
2  1956967696     sadness                Funeral ceremony...gloomy friday...   
3  1956967789  enthusiasm               wants to hang out with friends SOON!   
4  1956968416     neutral  @dannycastillo We want to trade with someone w...   

   category_id  
0            0  
1            1  
2            1  
3            2  
4            3  
     tweet_id   sentiment                                            content  \
0  1956967341       empty  <user> know listenin bad habit earlier started...   
1  1956967666     sadness             layin n bed headache ughhhhwaitin call   
2  1956967696     sadness                      funeral ceremonygloomy friday   
3  1956967789  enthusiasm                         wants hang friends soon !    
4  1956968416    

In [6]:
import numpy as np
# embeddings_text = open('glove50.txt')
WordToVec       = dict()
with open('./../../../glove.twitter.27B.50d.txt') as embeddings_text: 
    for line in embeddings_text:
        entry_array = line.split(" ")
        word        = entry_array[0] 
        word_vector = np.array(entry_array[1:],dtype='float32')
        WordToVec[word] = word_vector

In [7]:
print(WordToVec.get("<user>"))

[ 0.78704   0.72151   0.29148  -0.056527  0.31683   0.47172   0.023461
  0.69568   0.20782   0.60985  -0.22386   0.7481   -2.6208    0.20117
 -0.48104   0.12897   0.035239 -0.24486  -0.36088   0.026686  0.28978
 -0.10698  -0.34621   0.021053  0.54514  -1.0958   -0.274     0.2233
  1.0827   -0.029018 -0.84029   0.58619  -0.36511   0.34016   0.89615
  0.32757   0.24267   0.68404  -0.34374   0.13583  -2.2162   -0.42537
  0.46157   0.88626  -0.22014   0.025599 -0.38615   0.080107 -0.075323
 -0.61461 ]


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from keras import optimizers

Using TensorFlow backend.


In [28]:
max_features = 8000
tokenizer = Tokenizer(num_words=max_features,lower=True, split=' ')
tokenizer.fit_on_texts(df['content'].values)
print(tokenizer.document_count)
X = tokenizer.texts_to_sequences(df['content'].values)
X = pad_sequences(X)
X.shape
X[1]


40000


array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
       194,  79, 329, 197], dtype=int32)

In [10]:
# from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer(stop_words='english',max_features=8000)

# X_train_counts = count_vect.fit_transform(df['content'].values)
# print(X_train_counts.shape)

In [54]:
# vocab=len(tokenizer.word_index.items())
# print(vocab)
# print(tokenizer.word_index.items())

In [10]:
embedding_matrix = np.zeros((max_features, 50))

for word, index in tokenizer.word_index.items():
    if index > max_features-1:
        continue
    else:
        embedding_vector = WordToVec.get(word.lower())
#         print(embedding_vector)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [11]:
embedding_matrix[15]

array([-0.16140001,  0.22660001,  0.039414  ,  0.33642   , -0.62880999,
       -0.1035    ,  0.89679003,  0.72547001, -0.91096002,  0.066873  ,
       -0.88328999, -0.10026   , -3.98510003, -0.17095999, -0.77985001,
        0.74011999, -0.57288003, -0.23543   , -1.22300005,  0.83262002,
        0.15536   ,  0.75707   , -1.06980002,  0.75106001,  0.24905001,
        0.73247999, -0.13189   , -0.74685001,  1.55799997, -0.35084999,
       -0.41599   , -0.22845   ,  0.0049928 ,  0.57968998,  0.15524   ,
        0.41604999,  0.14427   ,  0.23541   ,  0.069321  ,  0.86312997,
       -2.20830011, -0.95704001,  0.57643998,  0.55662   ,  0.077452  ,
        0.10875   , -0.39614999, -0.81149   , -0.43867999,  0.063696  ])

In [12]:
embed_dim = 50
lstm_out = 64

adam = optimizers.Adam(lr=0.01)
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = Sequential()
# model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1], weights=[embedding_matrix], trainable=False))
# model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(50,activation='relu'))
model.add(Dense(13,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            400000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_1 (Dense)              (None, 13)                845       
Total params: 430,285
Trainable params: 30,285
Non-trainable params: 400,000
_________________________________________________________________
None


In [14]:
Y = pd.get_dummies(df['category_id']).values

print(X.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 0)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(40000, 30)
(36000, 30) (36000, 13)
(4000, 30) (4000, 13)


In [15]:
validation_size = 1000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

In [53]:
batch_size = 32
history=model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2,validation_data=(X_validate,Y_validate))

Train on 36000 samples, validate on 1000 samples
Epoch 1/20
 - 20s - loss: 1.7344 - acc: 0.4036 - val_loss: 1.9286 - val_acc: 0.3680
Epoch 2/20
 - 20s - loss: 1.7321 - acc: 0.4025 - val_loss: 1.9224 - val_acc: 0.3640
Epoch 3/20
 - 20s - loss: 1.7292 - acc: 0.4023 - val_loss: 1.9234 - val_acc: 0.3710
Epoch 4/20
 - 23s - loss: 1.7312 - acc: 0.4024 - val_loss: 1.9225 - val_acc: 0.3750
Epoch 5/20
 - 21s - loss: 1.7290 - acc: 0.4041 - val_loss: 1.9308 - val_acc: 0.3750
Epoch 6/20
 - 22s - loss: 1.7250 - acc: 0.4043 - val_loss: 1.9296 - val_acc: 0.3700
Epoch 7/20
 - 23s - loss: 1.7273 - acc: 0.4037 - val_loss: 1.9272 - val_acc: 0.3740
Epoch 8/20
 - 21s - loss: 1.7242 - acc: 0.4057 - val_loss: 1.9375 - val_acc: 0.3650
Epoch 9/20
 - 21s - loss: 1.7267 - acc: 0.4028 - val_loss: 1.9336 - val_acc: 0.3710
Epoch 10/20
 - 21s - loss: 1.7197 - acc: 0.4051 - val_loss: 1.9364 - val_acc: 0.3680
Epoch 11/20
 - 21s - loss: 1.7215 - acc: 0.4049 - val_loss: 1.9394 - val_acc: 0.3620
Epoch 12/20
 - 22s - loss

In [55]:

score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.88
acc: 0.37


In [56]:
 Y_pred=model.predict(X_test)
print("--Test matrices--")
print("accuracy")
print(accuracy_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1)))
print("f1_score")
print(f1_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1), average="macro"))
print("Precision_score")
print(precision_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1), average="macro"))
print("Recall_score")
print(recall_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1), average="macro"))
print("Confusion_matrix")
print(confusion_matrix(Y_test.argmax(axis=1),Y_pred.argmax(axis=1)))

print("--Train matrices--")
y_pred_train = model.predict(X_train)
print("accuracy")
print(accuracy_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1)))
print("f1_score")
print(f1_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1), average="macro"))
print("Precision_score")
print(precision_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1), average="macro"))
print("Recall_score")
print(recall_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1), average="macro"))
print("Confusion_matrix")
print(confusion_matrix(Y_train.argmax(axis=1),y_pred_train.argmax(axis=1)))

--Test matrices--
accuracy
0.36666666666666664
f1_score
0.18760687320409924
Precision_score
0.2901805241095606
Recall_score
0.18986316759934938
Confusion_matrix
[[  0   0   0  24  16   0   1   0   0   2   0   0   0]
 [  1  88   0  76 201   2  14   1   5  34   1   1   0]
 [  0   2   1  22  20   0   4   1   0   9   0   0   0]
 [  1  38   0 357 149   4  36   5   5  61   1   4   0]
 [  0  60   0 138 326   1  22   5  11  52   1   7   0]
 [  0  21   0  44  38   2  11   4   1  30   0   0   0]
 [  0  10   0  38  30   0 131   3   1  70   0   1   0]
 [  0   5   0  36  29   1  10   7   0  41   0   0   0]
 [  0  13   0  15  47   2   1   0  23   5   0   0   0]
 [  0  13   0  85  62   2  57  10   2 162   0   8   0]
 [  0   2   0   2   6   0   1   0   0   3   0   0   0]
 [  0   2   0  30  28   1   8   1   1  24   0   3   0]
 [  0   0   0   2   3   0   0   1   1   0   0   0   0]]
--Train matrices--


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy
0.4533333333333333
f1_score
0.2858105427837289
Precision_score
0.5612348774082779
Recall_score
0.26783772636265474
Confusion_matrix
[[  13   54    0  354  233    3   16   11   14   56    1    3    0]
 [   1 1553    0  710 1843   15  141   26   70  247    3   17    0]
 [   0   48    8  244  182    6   27    6    1  153    0    0    0]
 [   4  286    1 4754 1545   26  316   43   36  703    1   49    0]
 [   1  595    0 1408 4768   19  240   41   67  444    0   31    0]
 [   0   99    1  590  647  117  143   24   17  323    0   13    0]
 [   0  119    1  457  353   14 1822   26   10  641    0   25    0]
 [   0   44    1  414  285    7   91  227    7  520    0    9    0]
 [   0  126    0  151  463    3   12    6  388   27    0    4    0]
 [   0   86    4  891  560   19  555   88   12 2413    0   49    0]
 [   2   22    0   26   72    0    5    1    5    6   21    2    0]
 [   0   51    0  380  345    5  106   14    5  256    1  235    0]
 [   0    7    0   23   46    2    3    3  

In [57]:
def colorize(X,Y): #takes in X and Y as list of text strings
    emotions_dict =  {'empty':"\033[1;30;47m ", 'sadness':"\033[1;37;45m ", 'enthusiasm':"\033[1;33;42m ", 'neutral':"\033[1;37;40m ", 'worry':"\033[1;31;47m ", 'surprise':"\033[1;32;43m ", 'love':"\033[1;35;46m ", 'fun':"\033[1;33;42m ", 'hate':"\033[1;31;40m ", 'happiness':"\033[1;36;43m ", 'boredom':"\033[1;36;47m ", 'relief':"\033[1;33;44m ", 'anger':"\033[1;47;41m "}
    for i in range(len(X)):
        print(emotions_dict[Y[i]]+X[i])
        print(Y[i])

In [58]:
def analysizeSampleParaSentiment(sample_paragraph):
    lines=sample_paragraph.split(".")
    print(lines)
    df_2 = pd.DataFrame(lines, columns=['content'])
    df_2=removeIrrelvantInfo(df_2)
    print(df_2)
    X_sample = tokenizer.texts_to_sequences(df_2['content'].values)
    X_pad_sample = pad_sequences(X_sample, maxlen=30)
    Y_sample_pred=model.predict(X_pad_sample)
    Y_sample_pred_arr=Y_sample_pred.argmax(axis=1)
    Y_sample_pred_map=[]
    for pred in range(0,len(Y_sample_pred_arr)):
        Y_sample_pred_map.append(id_to_category[Y_sample_pred_arr[pred]])
    print(colorize(df_2['content'].values,Y_sample_pred_map))

In [59]:
#angry
sample_paragraph='I feel angry when I see some people throwing trash bags in public places. Having picnics at beaches and lakes is unenjoyable because of sickening smells. Plastic cups and cases are spattered on the green lawn though a lot of trash tanks spread in the garden'
analysizeSampleParaSentiment(sample_paragraph)

['I feel angry when I see some people throwing trash bags in public places', ' Having picnics at beaches and lakes is unenjoyable because of sickening smells', ' Plastic cups and cases are spattered on the green lawn though a lot of trash tanks spread in the garden']
                                             content
0  feel angry see people throwing trash bags publ...
1  picnics beaches lakes unenjoyable sickening sm...
2  plastic cups cases spattered green lawn though...
 feel angry see people throwing trash bags public places
hate
 picnics beaches lakes unenjoyable sickening smells
neutral
 plastic cups cases spattered green lawn though lot trash tanks spread garden
worry
None


In [60]:
#sad
sample_paragraph='a small smile settled onto my tear stained face. This is what everyone wanted, well then this is what they were getting. The thin blade settled on my skin  it had been there many a time'
analysizeSampleParaSentiment(sample_paragraph)

['a small smile settled onto my tear stained face', ' This is what everyone wanted, well then this is what they were getting', ' The thin blade settled on my skin  it had been there many a time']
                                      content
0  small smile settled onto tear stained face
1               everyone wanted, well getting
2           thin blade settled skin many time
 small smile settled onto tear stained face
love
 everyone wanted, well getting
neutral
 thin blade settled skin many time
worry
None


In [61]:
#enthusiasm
sample_paragraph='passion lets us engage deeply in things that matter deeply. When we are passionate, we have more energy for our pursuits. Passion heightens our awareness and mental acuity'
analysizeSampleParaSentiment(sample_paragraph)

['passion lets us engage deeply in things that matter deeply', ' When we are passionate, we have more energy for our pursuits', ' Passion heightens our awareness and mental acuity']
                                             content
0  passion lets us engage deeply things matter de...
1                        passionate, energy pursuits
2          passion heightens awareness mental acuity
 passion lets us engage deeply things matter deeply
neutral
 passionate, energy pursuits
neutral
 passion heightens awareness mental acuity
worry
None


In [62]:
#worry
sample_paragraph='We live in a society where the lines between good and bad have become blurred. And so we worry. We worry about a lot of things in our lives, most of which are a result of not fully understanding our situations'
analysizeSampleParaSentiment(sample_paragraph)

['We live in a society where the lines between good and bad have become blurred', ' And so we worry', ' We worry about a lot of things in our lives, most of which are a result of not fully understanding our situations']
                                             content
0         live society lines good bad become blurred
1                                              worry
2  worry lot things lives, result fully understan...
 live society lines good bad become blurred
worry
 worry
worry
 worry lot things lives, result fully understanding situations
neutral
None


In [63]:

sample_paragraph='I feel angry when I see some people throwing trash bags in public places. Having picnics at beaches and lakes is unenjoyable because of sickening smells. Plastic cups and cases are spattered on the green lawn though a lot of trash tanks spread in the garden'
lines=sample_paragraph.split(".")
print(lines)
df_2 = pd.DataFrame(lines, columns=['content'])
df_2=removeIrrelvantInfo(df_2)
print(df_2)


['I feel angry when I see some people throwing trash bags in public places', ' Having picnics at beaches and lakes is unenjoyable because of sickening smells', ' Plastic cups and cases are spattered on the green lawn though a lot of trash tanks spread in the garden']
                                             content
0  feel angry see people throwing trash bags publ...
1  picnics beaches lakes unenjoyable sickening sm...
2  plastic cups cases spattered green lawn though...


In [64]:
id_to_category

{0: 'empty',
 1: 'sadness',
 2: 'enthusiasm',
 3: 'neutral',
 4: 'worry',
 5: 'surprise',
 6: 'love',
 7: 'fun',
 8: 'hate',
 9: 'happiness',
 10: 'boredom',
 11: 'relief',
 12: 'anger'}

In [31]:
X_sample = tokenizer.texts_to_sequences(df_2['content'].values)
X_pad_sample = pad_sequences(X_sample, maxlen=30)

In [32]:
X_pad_sample

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   46,
        1758,   25,   77, 4331, 3653, 2689, 1073, 1300],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0, 2730],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 4283, 6623,
        4538,  754, 2425,   63,  228, 3653, 2818,  688]], dtype=int32)

In [33]:
Y_sample_pred=model.predict(X_pad_sample)
print(Y_sample_pred)

[[2.3380840e-02 1.8734878e-01 2.1010756e-03 6.5293901e-02 2.6555657e-01
  7.3320055e-03 1.2068903e-02 3.4220149e-03 4.2063975e-01 1.3144353e-03
  2.3809189e-03 1.2880785e-03 7.8727845e-03]
 [4.9127255e-02 1.0452608e-01 1.4385131e-02 4.0374064e-01 1.5300165e-01
  8.9618355e-02 2.5624309e-02 9.9109113e-03 3.4930322e-02 9.8344669e-02
  5.8089790e-04 1.3724314e-02 2.4855277e-03]
 [8.3512112e-02 7.4737094e-02 3.5744389e-03 1.8216166e-01 4.1276416e-01
  2.2407407e-02 6.5536499e-03 2.5987947e-02 9.3890510e-02 7.8625113e-02
  2.6966576e-04 1.1993238e-02 3.5230380e-03]]


In [38]:
Y_sample_pred_arr=Y_sample_pred.argmax(axis=1)

In [39]:
Y_sample_pred_map=[]

In [43]:
for pred in range(0,len(Y_sample_pred_arr)):
    print(id_to_category[Y_sample_pred_arr[pred]])
    Y_sample_pred_map.append(id_to_category[Y_sample_pred_arr[pred]])
print(Y_sample_pred_map)

hate
neutral
worry
['hate', 'neutral', 'worry']


In [46]:
colorize(df_2['content'].values,Y_sample_pred_map)

 feel angry see people throwing trash bags public places
hate
 picnics beaches lakes unenjoyable sickening smells
neutral
 plastic cups cases spattered green lawn though lot trash tanks spread garden
worry
